####잔차 연결  

***잔차 연결(residual connection)*** 이란 다음 그림처럼 블록이나 레이어 계산을 건너뛰는 경로를 하나 두는 것을 말합니다. 입력을 $x$, 이번 계산 대상 블록을 $F$라고 할 때 잔차 연결은 $F(x) + x$로 간단히 실현합니다.

<center><그림 3-49 잔차 연결(1)></center>

<p align="center"><img src="https://i.imgur.com/7DUovTQ.png">  

<center>출처 : ratsgo's NLPBOOK</center>



블록 계산이 계속될 때 잔차 연결을 두는 것은 제법 큰 효과가 있습니다. 다음 그림의 왼쪽처럼 블록 연산을 3번 수행하고 블록과 블록 사이에 잔차 연결을 모두 적용했다고 가정해 봅시다. 그렇다면 모델은 사실상 그림의 오른쪽처럼 계산하는 형태가 됩니다.

<center><그림 3-49 잔차 연결(2)></center>

<p align="center"><img src="https://i.imgur.com/UHVuX1X.jpg">  

<center>출처 : ratsgo's NLPBOOK</center>

그림에서 오른쪽을 보면 잔차 연결을 두지 않았을 때는 $f_1, f_2, f_3$을 연속으로 수행하는 경로 한 가지만 존재했으나, 잔차 연결을 블록마다 설정해둠으로써 모두 8가지의 새로운 경로가 생겼습니다. 다시 말해 모델이 다양한 관점에서 블록 계산을 수행하게 된다는 이야기입니다.

딥러닝 모델은 레이어가 많아지면 학습이 어려운 경향이 있습니다. 모델을 업데이트하기 위한 신호(그레이디언트)가 전달되는 경로가 길어지기 때문입니다. 잔차 연결은 모델 중간에 블록을 건너뛰는 경로를 설정함으로써 학습을 쉽게 하는 효과까지 거둘 수 있습니다.

####레이어 정규화  

***레이어 정규화(layer normalization)*** 란 미니 배치의 인스턴스($x$)별로 평균($E[x]$)을 빼주고 표준편차($\sqrt{E[x]}$)로 나눠 정규화를 수행하는 기법입니다. 레이어 정규화를 수행하면 학습이 안정되고 그 속도가 빨라지는 등의 효과가 있다고 합니다.  
다음 수식에서 $\beta$와 $\gamma$는 학습 과정에서 업데이트되는 가중치이며, $\epsilon$은 분모가 0이 되는 걸 방지하려고 더해주는 고정값(보통 1e-5로 설정)입니다.

<center><수식 3-27 레이어 정규화></center>

$$\mathbf{y}=\frac { \mathbf{x} - \mathop{\mathbb{E}} \left[ \mathbf{x} \right] }{ \sqrt { \mathop{\mathbb{V}} \left[ \mathbf{x} \right] +\epsilon } } *\gamma +\beta$$ 


레이어 정규화 수행 과정의 일부를 나타낸 것입니다. 배치의 첫 번째 데이터($x$=[1 2 3])의 평균과 표준편차는 각각 2, 0.8164인데요, 이 값들을 바탕으로 수식 3-27과 같은 정규화 수식을 계산하게 됩니다.

<center><그림 3-51 레이어 정규화 수행 과정 일부></center>

<p align="center"><img src="https://i.imgur.com/axo9eTU.png">  

<center>출처 : ratsgo's NLPBOOK</center>



그러면 코드로도 확인해 보겠습니다. 다음 코드는 파이토치로 레이어 정규화를 수행하는 역할을 합니다. 입력의 모양은 '배치 크기(2) $\times$ 피처의 자원 수(3)'가 되는데요, **torch.nn.LayerNorm(input.shape[-1])**이라는 코드는 위 그림에서 화살표 방향대로 레이어 정규화를 수행한다는 의미입니다.

In [1]:
#코드 3-8 레이어 정규화 예시
import torch
input = torch.tensor([[1.0, 2.0, 3.0], [1.0, 1.0, 1.0]])
m = torch.nn.LayerNorm(input.shape[-1])
output = m(input)

배치의 첫 번째 데이터를 수식 3-27에 따라 정규화하면 그 결과는 [-1.2247 0.0 1.2247]이 됩니다. 같은 방식으로 배치의 두 번째 데이터를 정규화하면 [0.0 0.0 0.0]이 됩니다.  
다음은 앞 코드의 **output**을 파이썬 콘솔에서 확인할 결과입니다.

In [2]:
output

tensor([[-1.2247,  0.0000,  1.2247],
        [ 0.0000,  0.0000,  0.0000]], grad_fn=<NativeLayerNormBackward0>)

앞서 보인 레이어 정규화 수식을 자세히 보면 평균을 빼고 표준편차로 나눈 결과에 $\gamma$를 곱하고 마지막으로 $\beta$를 더하는 걸 알 수 있습니다. 그런데 앞의 출력 결과를 보면 이 계산을 생략한 것 같은 인상을 주는군요.  
하지만 다음처럼 확인해 보면 이 의문이 풀립니다.

In [3]:
m.weight

Parameter containing:
tensor([1., 1., 1.], requires_grad=True)

In [4]:
m.bias

Parameter containing:
tensor([0., 0., 0.], requires_grad=True)

**m.weight**는 $\gamma$, **m.bias**는 $\beta$에 대응하는데요, 파이토치의 **LayerNorm** 객체는 이 두 값을 각각 1과 0으로 초기화합니다. 다시 말해 학습 초기 레이어 정규화 수행은 배치 인스턴스의 평균을 빼고 표준편차로 나눈 결과에 1을 곱한 후 0을 더한다는 이야기입니다.  
이후 학습 과정에서는 task(예를 들면 기계 번역)를 가장 잘 수행하는 방향으로 이 값들을 업데이트합니다.


### 모델 학습 기법
---  
여기서는 트랜스포머 모델의 학습 기법을 살펴봅니다.

####드롭아웃  
딥러닝 모델은 그 표현력이 아주 좋아서 학습 데이터 그 자체를 외워버릴 염려가 있습니다. 이를 ***과적합(overfitting)*** 이라고 합니다. ***드롭아웃(dropout)*** 은 이러한 과적합 현상을 방지하고자 뉴런의 일부를 확률적으로 0으로 대치하여 계산에서 제외하는 기법입니다.

<center><그림 3-52 일반적인 뉴럴 네트워크(왼쪽)에 드롭아웃을 적용한 예(오른쪽)></center>

<p align="center"><img src="https://i.imgur.com/pKUE70B.png">  

<center>출처 : ratsgo's NLPBOOK</center>

다음은 드롭아웃을 구현한 파이토치 코드입니다. **torch.nn.Dropout** 객체는 뉴런별로 드롭아웃을 수행할지 말지를 확률적으로 결정하는데요, **p=0.2**는 드롭아웃 수행 비율이 평균적으로 20%가 되게끔 하겠다는 이야기입니다.

In [5]:
#코드 3-9 드롭아웃
import torch
m = torch.nn.Dropout(p=0.2)
input = torch.randn(1, 10)
output = m(input)

다음은 앞 코드의 **input**과 **output**을 파이썬 콘솔에서 확인한 결과입니다. 드롭아웃 수행 결과 **input** 뉴런 가운데 8번째, 10번째가 0으로 대치되었음을 확인할 수 있습니다.

In [6]:
input

tensor([[ 0.3707, -1.2887,  0.8247,  0.6030,  1.2650, -0.3377,  1.2830, -0.9661,
         -1.4006, -0.3601]])

In [7]:
output

tensor([[ 0.4633, -1.6108,  1.0309,  0.0000,  1.5812, -0.0000,  1.6038, -1.2076,
         -0.0000, -0.4501]])

참고로 **torch.nn.Dropout**은 안정적인 학습을 위해 각 요솟값에 **1/(1-p)**를 곱하는 역할도 수행합니다. 코드 3-9에서 **p**를 0.2로 설정해 두었으므로 드롭아웃 적용으로 살아남은 요솟값 각각에 1.25를 곱하는 셈입니다. 이에 1.0573는 **torch.nn.Dropout** 수행 후 1.3217로, 0.1351은 0.1689로 변환됐습니다.

트랜스포머 모델에서 드롭아웃은 입력 임베딩과 최초 블록 사이, 블록과 블록 사이, 마지막 블록과 출력층 사이 등에 적용합니다. 드롭아웃 비율은 10%($p=0.1$)로 설정하는 것이 일반적입니다.  
드롭아웃은 학습 과정에만 적용하고 학습이 끝나고 나서 인퍼런스 과정에서는 적용하지 않습니다.

#### 아담 옵티마이저  

딥러닝 모델 학습은 모델 출력과 정답 사이의 ***오차(error)*** 를 최소화하는 방향을 구하고 이 방향에 맞춰 모델 전체의 ***파라미터(parameter)*** 들을 업데이트하는 과정입니다. 이때 오차를 ***손실(loss)***, 오차를 최소화하는 방향을 ***그레이디언트(gradient)*** 라고 합니다. 그리고 오차를 최소화하는 과정을 ***최적화(optimization)*** 라고 합니다.

파라미터란 행렬, 벡터, 스칼라 따위의 모델 구성 요소입니다. 이 값들은 학습 데이터로 구합니다. 예를 들어 대한민국 남성의 키를 '정규 분포'라는 모델로 나타낸다고 가정한다면 대한민국 남성 키의 평균과 표준편차가 이 모델의 파라미터가 됩니다.

딥러닝 모델을 학습하려면 우선 오차부터 구해야 합니다. 오차를 구하려면 현재 시점의 모델에 입력을 넣어봐서 처음부터 끝까지 계산해 보고 정답과 비교해야 합니다. 이처럼 오차를 구하려고 모델 처음부터 끝까지 순서대로 계산해 보는 과정을 ***순전파(forward propagation)*** 라고 합니다.

오차를 구했다면 오차를 최소화하는 최초의 그레이디언트를 구할 수 있습니다. 이는 ***미분(derivative)*** 으로 구합니다. 이후 미분의 ***연쇄 법칙(chain rule)*** 에 따라 모델 파라미터별 그레이디언트 역시 구할 수 있습니다. 이 과정은 순전파의 역순으로 순차적으로 수행되는데요, 이를 ***역전파(backpropagation)*** 라고 합니다. 다음 그림은 순전파의 역전파를 개념적으로 나타낸 그림입니다.

<center><그림 3-53 순전파와 역전파></center>

<p align="center"><img src="https://i.imgur.com/b551jfH.png">  

<center>출처 : ratsgo's NLPBOOK</center>

모델 파라미터를 업데이트하는 과정, 즉 학습 과정은 미니 배치 단위로 이뤄지는데요, 이는 눈을 가린 상태에서 산등성이를 한 걸음씩 내려가는 과정에 비유할 수 있습니다. 내가 지금 있는 위치에서 360도 모든 방향에 대해 한 발 한 발 내디뎌 보고 가장 경사가 급한 쪽으로 한 걸음씩 내려가는 과정을 반복하는 것입니다.

모델 파라미터를 업데이트할 때(산등성이를 내려갈 때) 중요한 것은 방향과 보폭일 것입니다. 이는 최적화 도구의 도움을 받는데요, 트랜스포머 모델이 쓰는 최적화 도구가 바로 ***아담 옵티마이저(Adam optimizer)*** 입니다. 아담 옵티마이저는 오차를 줄이는 성능이 좋아서 트랜스포머 말고도 다양한 모델 학습에 널리 쓰이고 있습니다.

아담 옵티마이저의 핵심 동작 원리는 방향과 보폭을 적절하게 정해주는 겁니다. 방향을 정할 때는 현재 위치에서 경사가 가장 급한 쪽으로 내려가되, 여태까지 내려오던 관성(방향)을 일부 유지하도록 합니다. 보폭의 경우 안 가본 곳은 성큼 빠르게 걸어 훑고, 많이 가본 곳은 갈수록 보폭을 줄여 세밀하게 탐색하는 방식으로 정합니다. 

다음은 아담 옵티마이저를 사용하는 파이토치 코드입니다. 최초의 보폭, 즉 ***러닝 메이트(learning rate)*** 를 정해주면 아담 옵티마이저가 최적화 대상 파라미터들(**model.parameters()**)에 방향과 보폭을 정해줍니다.